# Initial Exploration

In [ ]:
import os
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

## Orders

In [ ]:
# Create a list of DataFrames from the orders CSV files
orders_dir = '../data/raw/orders'
csv_files = [f for f in os.listdir(orders_dir) if f.endswith('.csv')]

dataframes = []
for file in csv_files:
    file_path = os.path.join(orders_dir, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Combine dataframes into one
df = pd.concat(dataframes, ignore_index=True)
df.head()

In [ ]:
print(f"Rows: {df.shape[0]}\nColumns: {df.shape[1]}")

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# Convert date columns to datetime

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Fulfillment Date'] = pd.to_datetime(df['Fulfillment Date'], format='%m/%d/%Y, %I:%M %p')

### Data Quality Checks
- [x] Investigate missing values
- [x] Identify static columns
- [x] Check for duplicates
- [x] Validate data types
- [x] Spot outliers

##### Missing values and static columns

In [ ]:
# Identify empty or static columns

for col in df.columns:
    unique_values = set(df[col].dropna().unique())
    if len(unique_values) == 0:
        print(f"Column '{col}' is empty.")
    elif len(unique_values) == 1:
        print(f"Column '{col}' has a single unique value: {unique_values.pop()}")

In [ ]:
# List distinct Recipient Country values

df['Recipient Country'].unique()

In [ ]:
zz_country = df[df['Recipient Country'] == 'ZZ']
zz_country

In [ ]:
len(zz_country)

In [ ]:
# View missing value percentages

df.isna().sum().div(len(df))*100

* `Currency`, `Fulfillment Location`, and `Recipient Region` have a single unique value.
* `Order Shipping Price`, `Order Refunded Amount`, and `Item SKU` are empty.
* `Recipient Country` has 359 rows with the value `ZZ`.
* Geographic details such as `Recipient Address` and `Recipient Postal Code` are missing greater than **89%** of their data.

#### Null values

In [ ]:
# View rows with all null values

df[df.isna().all(axis=1)]

#### Duplicates

In [ ]:
# Check for duplicates

duplicates = df.duplicated()
df[duplicates]

In [ ]:
# Check duplicate 'Armen 59-07' rows

df[(df['Order'] == 'Armen 59-07') & (df['Order Date'] == '2024/08/31')]

In [ ]:
# Check duplicate Troy Issac rows

df[(df['Order'] == 'Troy Issac') & (df['Order Date'] == '2024/12/19')]

Each order is split into multiple rows, one for each menu item. Menu items are not grouped together however. For example, one order can have multiple rows with `CRINKLE FRIES` as the `Item name`. This implies that if items are part of separate combos or groupings, they are listed separately.

#### Outliers

In [ ]:
def calculate_iqr(df: pd.DataFrame, col: str) -> tuple:
    """
    Calculate the Interquartile Range (IQR) for a given column.
    
    Args:
        col (str): The name of the column to calculate IQR for.
        
    Returns:
        tuple: A tuple containing the lower and upper bounds for outliers.
    """
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    return lower_bound, upper_bound


##### Order Total

In [ ]:
# Identify outliers in Order Total

plt.title('Distribution of Order Total', fontsize=16, fontweight='bold')
sns.histplot(data=df, x='Order Total', color='skyblue')
plt.xlabel('Order Total ($)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
lower_bound, upper_bound = calculate_iqr(df=df, col='Order Total')

plt.axvline(x=df['Order Total'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(x=df['Order Total'].median(), color='green', linestyle='--', label='Median')
plt.axvline(x=lower_bound, color='orange', linestyle='-.', label='Lower Bound') if lower_bound > 0 else None
plt.axvline(x=upper_bound, color='orange', linestyle='-.', label='Upper Bound')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Print outlier information

num_order_total_outliers = len(df[df['Order Total'] > upper_bound])
print(f"Number of outliers in 'Order Total': {num_order_total_outliers}")
print(f"Percentage of total dataset: {num_order_total_outliers / len(df) * 100:.2f}%")

In [ ]:
# View Order Total outtliers

order_total_outliers = df[df['Order Total'] > upper_bound].sort_values(by='Order Total', ascending=False)
order_total_outliers

It appears that outliers in `Order Total` are simply large orders. At a cursory investigation there does not seem to be data entry mistakes or suspicious activity.

##### Item Price

In [ ]:
# Identify outliers in Item Price

plt.title('Distribution of Item Price', fontsize=16, fontweight='bold')
sns.histplot(data=df, x='Item Price', color='skyblue')
plt.xlabel('Item Price ($)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

lower_bound, upper_bound = calculate_iqr(df=df, col='Item Price')

plt.axvline(x=df['Item Price'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(x=df['Item Price'].median(), color='green', linestyle='--', label='Median')
plt.axvline(x=lower_bound, color='orange', linestyle='-.', label='Lower Bound') if lower_bound > 0 else None
plt.axvline(x=upper_bound, color='orange', linestyle='-.', label='Upper Bound')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# View Item Price outlier information

num_item_price_outliers = len(df[df['Item Price'] > upper_bound])
print(f"Number of outliers in 'Item Price': {num_item_price_outliers}")
print(f"Percentage of total dataset: {num_item_price_outliers / len(df) * 100:.2f}%")

In [ ]:
# List Item Price outliers

item_price_outliers = df[df['Item Price'] > upper_bound].sort_values(by='Item Price', ascending=False)
item_price_outliers

In [ ]:
# List Item Price outlier Items

item_price_outliers['Item Name'].unique()

Outliers in `Item Price` are catering packages, platters, bundles, and more expensive combos.

##### Item Options Total Price

In [ ]:
# Identify outliers in Item Options Total Price

plt.title('Distribution of Item Options Total Price', fontsize=16, fontweight='bold')
sns.histplot(data=df, x='Item Options Total Price', color='skyblue')
plt.xlabel('Item Options Total Price ($)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

lower_bound, upper_bound = calculate_iqr(df=df, col='Item Options Total Price')

plt.axvline(x=df['Item Options Total Price'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(x=df['Item Options Total Price'].median(), color='green', linestyle='--', label='Median')
plt.axvline(x=lower_bound, color='orange', linestyle='-.', label='Lower Bound') if lower_bound > 0 else None
plt.axvline(x=upper_bound, color='orange', linestyle='-.', label='Upper Bound')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
item_options_total_price_outliers = df[df['Item Options Total Price'] > upper_bound].sort_values(by='Item Options Total Price', ascending=False)
item_options_total_price_outliers

In [ ]:
# Print number of rows shared by all price outliers

merged_df = pd.merge(left=item_price_outliers, right=item_options_total_price_outliers, left_index=True, right_index=True, how='inner', suffixes=('_item_price', '_item_options_total_price'))
merged_df = pd.merge(left=merged_df, right=order_total_outliers, left_index=True, right_index=True, how='inner', suffixes=('', '_order_total'))

print(f"Number of merged outliers: {len(merged_df)}")

Rows which have a high `Item Price` also have a high `Item Options Total Price` and `Order Total`, signaling a relationship between these features.

### Business Logic Validation

- [x] Investigate tax calculation
- [x] Confirm total calculated correctly from subtotal
- [x] Check for canceled/voided/refunded orders

##### Total Calculation

In [ ]:
# Inspect rows where Order Subtotal and Order Tax Total don't add up to Order Total

tax_rate_valid = np.isclose((df['Order Subtotal'] + df['Order Tax Total']), df['Order Total'])
df[~tax_rate_valid].head()

It is not apparently cleaer why `Order Tax Total` and `Order Subtotal` do not add up to `Order Total` in all rows. Possible reasons could include added fees or tips, whose information is missing in this dataset.

#### Tax Calculation

In [ ]:
# Calculate effective tax rates

tax_rate_valid_df = df[tax_rate_valid]
(tax_rate_valid_df['Order Tax Total']/tax_rate_valid_df['Order Subtotal']).round(decimals=2).unique()

In [ ]:
# Count occurrences of each tax rate

df['Tax Rate'] = (df['Order Tax Total']/df['Order Subtotal']).round(decimals=2)
df.groupby(by='Tax Rate').size()

In [ ]:
df[df['Tax Rate'] == 0.05]

The statewide tax rate in California is **7.25%**. In Los Angeles, the combined sales tax rate (state and local) is **9.50%**. While a majority of the orders are between 9-10%, there are a small amount with a much lower tax rate.

#### Canceled/Voided/Refunded orders

In [ ]:
df['Fulfillment Status'].unique()

In [ ]:
# View cancelled orders

df[df['Fulfillment Status'] == 'Canceled']

In [ ]:
# Generate pseudo order id to be able to group orders

df['pseudo_order_id'] = df['Order Name'].str.split(' ').str[0] + '_' + df['Order Date'].astype(str)
df['pseudo_order_id']

In [ ]:
# Calculate number of canceled orders

cancelled_orders = df[df['Fulfillment Status'] == 'Canceled']
num_canceled_orders = cancelled_orders['pseudo_order_id'].nunique()

print(f"Number of canceled orders: {num_canceled_orders}")

In [ ]:
# View New orders

df[df['Fulfillment Status'] == 'New']

In [ ]:
# View orders with no fulfillment status

null_fulfillment_status = df[df['Fulfillment Status'].isnull()]
null_fulfillment_status

In [ ]:
null_fulfillment_status.info()

In [ ]:
# Investigate Order Name for orders with no fulfillment status

null_fulfillment_status['Order Name'].unique()

In [ ]:
# Count fulfillment types including null values

df['Fulfillment Type'].value_counts(dropna=False)

In [ ]:
# List Channels of orders with no fulfillment status

null_fulfillment_status['Channels'].unique()

In [ ]:
# List refunded orders

df[df['Order Refunded Amount'] > 0]

* There were a total of **5** cancelled orders.
* **2** orders were listed as **New**. These were probably in progress at the time of capturing the data.
* **826** rows have no `Fulfillment Status` listed.
    * These orders have primarily numeric `Order Name`.
    * They are also missing `Fulfillment Type`.
    * The single `Channel` for these rows is `BELLY RUBB - BBQ Ribs to Go & Catering`
* There are no refunded orders. This probably means that `Order Refunded Amount` does not apply when `Fulfillment Status` is `Canceled`.